## Introduction to Brian2

- Ideal for integrate-and-fire models, non-linear channel dynamics
- Multi-compartmental models (although not as popular)
- Not made for rate-based models.

**Strong points**
- Concise language
- Free
- Versatile and flexible
- Code generation: standalone mode for maximal performance (Brian's high-performance mode that turns whole simulation into a single compiled file. Faster.


**Weak points**
- Brian provides no support for cluster computations and limited support for multicore computers. Not made for big networks with lots of synapses (~1million).
- Code generation: standalone mode (you cannot interleave Brian code with Python code).


**Brian specificity**
- Mathematical notation, physical units

In [ ]:
from brian2 import *

a = 10*mV
b = 2*volt
c = a+b
print(c)

### Further material about Brian
- briansimulator.org
- brian2.readthedocs.org
- briansupport@googlegroups.com
- groups.google.com/forum/#!forum/briansupport
- github.com/brian-team/brian2

## Brian tutorial part 1: models of spiking neurons

### Defining a simple neuron model dynamics

<center>$C\frac{dV}{dt} = g_L(E_L - V) + I$</center>
with $C=100 pF$, 
$E_L=-70 mV$,  
$g_L=10 nS$ and 
$I = 1 nA$

In [ ]:
# Brian models are defined as strings
# Each variable definition in the model needs to state the physical dimensions





We now inject a periodic current.

<center>$I = 1 + 1.25 sin(2\pi f t)$</center>


## Brian tutorial part 2: synapses

In [ ]:
# Synapses objects take as arguments the source and target neuron group





### More  complex network connectivity

In [ ]:
# 1) no auto-connections, 
# 2) network with only 20% recurrency, 
# 3) synaptic weights different for different synapses




#def visualise_connectivity(S):
#    Ns = len(S.source)
#    Nt = len(S.target)
    
#    subplot(121)
#    scatter(zeros(Ns), arange(Ns), color='k')
#    scatter(ones(Nt), arange(Nt), color='k')
#    for i,j in zip(S.i, S.j):
#        plot([0,1], [i,j], color='k')
        
#    subplot(122)
#    scatter(S.i, S.j, color='k')
    
#figure()
#visualise_connectivity(synapse)

### STDP

$\Delta w = \sum_{pre}\sum_{post} W(t_{post}-t_{pre})$

with $W(\Delta t) = A_{pre}exp(-\Delta t/\tau_{pre})$ if $\Delta t >0$ and $W(\Delta t) = A_{post}exp(-\Delta t/\tau_{post})$ if $\Delta t <0$

In [ ]:
tau_stdp = 20*ms
A_pre = 0.01
A_post = -A_pre*1.05
delta_t = linspace(-50,50,100)*ms
W = where(delta_t>0, A_pre*exp(-delta_t/tau_stdp), A_post*exp(delta_t/tau_stdp))

plot(delta_t/ms, W)

Define new variables $a_{pre}$ and $a_{post}$ which are traces of pre- and post-synaptic activity:

$\tau_{pre} \frac{d}{dt}a_{pre} = -a_{pre}$  
$\tau_{post} \frac{d}{dt}a_{post} = -a_{post}$  

When a presynaptic spike occurs, the presynaptic trace is updated and the weight is modified according to the rule:

$a_{pre} \rightarrow a_{pre} + A_{pre}$, $w \rightarrow w + a_{post}$  
  
When a postsynaptic spike occurs:

$a_{post} \rightarrow a_{post} + A_{post}$, $w \rightarrow w + a_{pre}$ 

In [ ]:
start_scope()

taupre = taupost = 20*ms
wmax = 0.01
Apre = 0.01
Apost = -Apre*taupre/taupost*1.05

G = NeuronGroup(1, 'v:1', threshold='v>1', reset='v=0')

syn_model = Synapses(G,G, '''   
                           w:1
                           dapre/dt = -apre/taupre : 1 (event-driven)
                           dapost/dt = -apost/taupost : 1 (event-driven)
                           ''',
                    on_pre= '''
                            v_post +=w
                            apre += Apre
                            w = clip(w+apost, 0, wmax)
                    ''',
                    on_post = '''
                            apost += Apost
                            w = clip(w+apre, 0, wmax)
                    ''')

#  (event-driven) after the definitions of apre and apost. 
# although these two variables evolve continuously over time, 
# Brian should only update them at the time of an event (a spike). 
# This is because we don’t need the values of apre and apost except at spike times, 
# and it is more efficient to only update them when needed.

Defining synapses: 3 synaptic variables
(event-driven): Brian only updates variables apre, apost at the time of an event (spike). It is more efficient.
clip(x,low,high) : clamp variable x between low and high.
on_post=... argument: gives statement to calculate when a post-synaptic neuron fires.


In [ ]:
start_scope()

taupre = taupost = 20*ms
wmax = 0.01
Apre = 0.01
Apost = -Apre*taupre/taupost*1.05

G = NeuronGroup(2, 'v:1', threshold='t>(1+i)*10*ms', refractory=100*ms)

S = Synapses(G, G,
             '''
             w : 1
             dapre/dt = -apre/taupre : 1 (clock-driven)      
             dapost/dt = -apost/taupost : 1 (clock-driven)
             ''',
             on_pre='''
             v_post += w
             apre += Apre
             w = clip(w+apost, 0, wmax)
             ''',
             on_post='''
             apost += Apost
             w = clip(w+apre, 0, wmax)
             ''', method='linear')

S.connect(i=0, j=1)
M = StateMonitor(S, ['w', 'apre', 'apost'], record=True)

run(30*ms)

figure(figsize=(4, 8))
subplot(211)
plot(M.t/ms, M.apre[0], label='apre')
plot(M.t/ms, M.apost[0], label='apost')
legend()
subplot(212)
plot(M.t/ms, M.w[0], label='w')
legend(loc='best')
xlabel('Time (ms)');

## Brian tutorial part 3: some more complex examples

Changing parameters/variables during simulations.

**Example 1:** How does the firing rate of a LIF neuron driven by Poisson spiking neurons changes with its membrane time constant?

In [ ]:
# How to create a group of Poisson neurons


In the above simulation, both the tau and the number of spikes is changing in each simulation. To change this, we can run the Poisson group once, store its spikes, and then create a SpikeGeneratorGroup that will output those stored spikes at each simulation.

You can see that now there is much less noise and it increases monotonically because the input spikes are the same each time, meaning we are seeing the effect of the time constant alone, not the random spikes.

**Example 2:** Changing randomly the amplitude of a current acting on HH neuron every 10ms.

In [ ]:
start_scope()

# Parameters
area = 20000*umetre**2
Cm = 1*ufarad*cm**-2 * area
gl = 5e-5*siemens*cm**-2 * area
El = -65*mV
EK = -90*mV
ENa = 50*mV
g_na = 100*msiemens*cm**-2 * area
g_kd = 30*msiemens*cm**-2 * area
VT = -63*mV

# The model
eqs_HH = '''
dv/dt = (gl*(El-v) - g_na*(m*m*m)*h*(v-ENa) - g_kd*(n*n*n*n)*(v-EK) + I)/Cm : volt
dm/dt = 0.32*(mV**-1)*(13.*mV-v+VT)/
    (exp((13.*mV-v+VT)/(4.*mV))-1.)/ms*(1-m)-0.28*(mV**-1)*(v-VT-40.*mV)/
    (exp((v-VT-40.*mV)/(5.*mV))-1.)/ms*m : 1
dn/dt = 0.032*(mV**-1)*(15.*mV-v+VT)/
    (exp((15.*mV-v+VT)/(5.*mV))-1.)/ms*(1.-n)-.5*exp((10.*mV-v+VT)/(40.*mV))/ms*n : 1
dh/dt = 0.128*exp((17.*mV-v+VT)/(18.*mV))/ms*(1.-h)-4./(1+exp((40.*mV-v+VT)/(5.*mV)))/ms*h : 1
I : amp
'''

group = NeuronGroup(1, eqs_HH,
                    threshold='v > -40*mV',
                    refractory='v > -40*mV',
                    method='exponential_euler')
group.v = El
statemon = StateMonitor(group, 'v', record=True)
spikemon = SpikeMonitor(group, 'v')

figure(figsize=(9, 4))
for l in range(5):
    group.I = rand()*50*nA
    run(10*ms)
    axvline(l*10, ls='--', c='k')
axhline(El/mV, ls='-', c='lightgray', lw=3)
plot(statemon.t/ms, statemon.v[0]/mV, '-b')
plot(spikemon.t/ms, spikemon.v/mV, 'ob')
xlabel('Time (ms)')
ylabel('v (mV)');

Have to call the run on a lop over multiple simulations is not very efficient - takes a long time. An alternative is to replace the loop with the 'run_regularly' function.

In [ ]:
start_scope()
group = NeuronGroup(1, eqs_HH,
                    threshold='v > -40*mV',
                    refractory='v > -40*mV',
                    method='exponential_euler')
group.v = El
statemon = StateMonitor(group, 'v', record=True)
spikemon = SpikeMonitor(group, variables='v')

# we replace the loop with a run_regularly
group.run_regularly('I = rand()*50*nA', dt=10*ms)
run(50*ms)

figure(figsize=(9, 4))
# we keep the loop just to draw the vertical lines
for l in range(5):
    axvline(l*10, ls='--', c='k')
axhline(El/mV, ls='-', c='lightgray', lw=3)
plot(statemon.t/ms, statemon.v[0]/mV, '-b')
plot(spikemon.t/ms, spikemon.v/mV, 'ob')
xlabel('Time (ms)')
ylabel('v (mV)');

Now let's extend this example to run over multiple neurons, each with a different capacitance to see how that affects the behavior of the cell.

In [ ]:
start_scope()
N = 3
eqs_HH_2 = '''
dv/dt = (gl*(El-v) - g_na*(m*m*m)*h*(v-ENa) - g_kd*(n*n*n*n)*(v-EK) + I)/C : volt
dm/dt = 0.32*(mV**-1)*(13.*mV-v+VT)/
    (exp((13.*mV-v+VT)/(4.*mV))-1.)/ms*(1-m)-0.28*(mV**-1)*(v-VT-40.*mV)/
    (exp((v-VT-40.*mV)/(5.*mV))-1.)/ms*m : 1
dn/dt = 0.032*(mV**-1)*(15.*mV-v+VT)/
    (exp((15.*mV-v+VT)/(5.*mV))-1.)/ms*(1.-n)-.5*exp((10.*mV-v+VT)/(40.*mV))/ms*n : 1
dh/dt = 0.128*exp((17.*mV-v+VT)/(18.*mV))/ms*(1.-h)-4./(1+exp((40.*mV-v+VT)/(5.*mV)))/ms*h : 1
I : amp
C : farad
'''
group = NeuronGroup(N, eqs_HH_2,
                    threshold='v > -40*mV',
                    refractory='v > -40*mV',
                    method='exponential_euler')
group.v = El
# initialise with some different capacitances
group.C = array([0.8, 1, 1.2])*ufarad*cm**-2*area
statemon = StateMonitor(group, variables=True, record=True)
# we go back to run_regularly
group.run_regularly('I = rand()*50*nA', dt=10*ms)
run(50*ms)
figure(figsize=(9, 4))
for l in range(5):
    axvline(l*10, ls='--', c='k')
axhline(El/mV, ls='-', c='lightgray', lw=3)
plot(statemon.t/ms, statemon.v.T/mV, '-')
xlabel('Time (ms)')
ylabel('v (mV)');

Injected current is different for all the different neurons.

In [ ]:
plot(statemon.t/ms, statemon.I.T/nA, '-')
xlabel('Time (ms)')
ylabel('I (nA)');

run_regularly is interpreted as being run separately for each neuron, and since I is a parameter that is randomly chosen at each run, it is different for each neuron. We can fix this by making I into a shared variable, meaning that it has the same value for each neuron.

In [ ]:
start_scope()
N = 3
eqs_HH_3 = '''
dv/dt = (gl*(El-v) - g_na*(m*m*m)*h*(v-ENa) - g_kd*(n*n*n*n)*(v-EK) + I)/C : volt
dm/dt = 0.32*(mV**-1)*(13.*mV-v+VT)/
    (exp((13.*mV-v+VT)/(4.*mV))-1.)/ms*(1-m)-0.28*(mV**-1)*(v-VT-40.*mV)/
    (exp((v-VT-40.*mV)/(5.*mV))-1.)/ms*m : 1
dn/dt = 0.032*(mV**-1)*(15.*mV-v+VT)/
    (exp((15.*mV-v+VT)/(5.*mV))-1.)/ms*(1.-n)-.5*exp((10.*mV-v+VT)/(40.*mV))/ms*n : 1
dh/dt = 0.128*exp((17.*mV-v+VT)/(18.*mV))/ms*(1.-h)-4./(1+exp((40.*mV-v+VT)/(5.*mV)))/ms*h : 1
I : amp (shared) # everything is the same except we've added this shared
C : farad
'''
group = NeuronGroup(N, eqs_HH_3,
                    threshold='v > -40*mV',
                    refractory='v > -40*mV',
                    method='exponential_euler')
group.v = El
group.C = array([0.8, 1, 1.2])*ufarad*cm**-2*area
statemon = StateMonitor(group, 'v', record=True)
group.run_regularly('I = rand()*50*nA', dt=10*ms)
run(50*ms)
figure(figsize=(9, 4))
for l in range(5):
    axvline(l*10, ls='--', c='k')
axhline(El/mV, ls='-', c='lightgray', lw=3)
plot(statemon.t/ms, statemon.v.T/mV, '-')
xlabel('Time (ms)')
ylabel('v (mV)');